In [22]:
import pandas as pd
import os, sys

class data_cleanup():
    def __init__(self):
        # filename should be the same as the project ID (e.g. KICH)
        # filename = sys.argv[0]
        filename = 'KICH'
        file_dict = self.get_dir_list(filename)
        df = self.combine_df(file_dict)
        df.to_csv(filename + '.tsv', sep='\t')

    def get_dir_list(self, filename):
        file_dict = {'CANCER': [], 'NORMAL': []}
        for file in ['CANCER', 'NORMAL']:
            new_filename1 = filename + '/' + file + '/' + os.listdir(filename + '/' + file)[0]
            dir = os.listdir(new_filename1)
            for i in range(1, 10):
                new_filename2 = new_filename1 + '/' + dir[i] + '/'
                new_filename2 += os.listdir(new_filename2)[0]
                file_dict[file].append(new_filename2)
        return file_dict

    def read_file(self, filepath):
        df = pd.read_csv(filepath, sep='\t', skiprows=[0,2,3,4,5], index_col=False)
        components = filepath.split('/')
        df['sample_type'] = components[1]
        df['tumor_name'] = components[0]
        return df

    def combine_df(self, file_dict):
        df = pd.DataFrame()
        for key in ['CANCER', 'NORMAL']:
            for dir in file_dict[key]:
                df = pd.concat([df, self.read_file(dir)])
        return df

data_cleanup()


TypeError: combine_df() missing 1 required positional argument: 'file_dict'

In [23]:
df = pd.read_csv('KICH.tsv', sep='\t')

In [24]:
df.head(10)

,Unnamed: 0,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded,sample_type,tumor_name
0,0,ENSG00000000003.15,TSPAN6,protein_coding,1107,577,530,8.2367,5.1704,7.8826,CANCER,KICH
1,1,ENSG00000000005.6,TNMD,protein_coding,0,0,0,0.0000,0.0000,0.0000,CANCER,KICH
2,2,ENSG00000000419.13,DPM1,protein_coding,923,469,454,25.8092,16.2010,24.6997,CANCER,KICH
3,3,ENSG00000000457.14,SCYL3,protein_coding,190,145,176,0.9317,0.5848,0.8916,CANCER,KICH
4,4,ENSG00000000460.17,C1orf112,protein_coding,47,104,82,0.2657,0.1668,0.2543,CANCER,KICH
5,5,ENSG00000000938.13,FGR,protein_coding,296,147,150,2.9539,1.8542,2.8269,CANCER,KICH
6,6,ENSG00000000971.16,CFH,protein_coding,332,154,178,1.4042,0.8814,1.3438,CANCER,KICH
7,7,ENSG00000001036.14,FUCA2,protein_coding,1314,896,923,15.7151,9.8648,15.0395,CANCER,KICH
8,8,ENSG00000001084.13,GCLC,protein_coding,1154,647,655,4.5194,2.8369,4.3251,CANCER,KICH
9,9,ENSG00000001167.14,NFYA,protein_coding,556,310,295,4.9240,3.0909,4.7123,CANCER,KICH
